In [72]:
!nvidia-smi

Tue Nov  3 01:19:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [73]:
pip install transformers

In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel

In [75]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [76]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [77]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [78]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(float(n))
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [79]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")


In [80]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))


trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)

tokenized_text = []
for i in trainWords:
    sent = ""
    for j in i:
        if sent == "":
            sent += j
        else:
            sent = sent + " " + j
    tokens = tokenizer.tokenize(sent)
    tokenized_text.append(tokens)

    

# Encoding each word of the sentence and appending to a list
en=[]
for i in tokenized_text:
  en1 = []
  for j in i:
    token_ids = en1.append(tokenizer.encode_plus(j, add_special_tokens=False, return_attention_mask=False, return_tensors='pt'))
  en.append(en1)
  en1 = []
  
# Getting the input id's only for the list and passing them to the model 
words = []  
for d in en:
    words2 = []
    for i in d:
        words2.append(i['input_ids'])
    words.append(words2)
    words2 = []     

In [81]:
class BertBinaryClassifier(nn.Module):
    def __init__(self):
        super(BertBinaryClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('nghuyong/ernie-2.0-large-en')
        self.linear = nn.Linear(1024, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, pooled_output = self.bert(tokens)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        proba = proba.data.tolist()
        return proba

In [82]:
bert_clf = BertBinaryClassifier()
optimizer = optim.Adamax(bert_clf.parameters(), lr=0.0001)
bert_clf.train()
token_answers = []
for epoch_num in range(10):
    print("epoch_num ---->{}".format(epoch_num))
    count = 0
    for batch_data, batch_probs in zip(words, trainLabels):
        bert_clf.zero_grad()
        count = count + 1
        answers_temp = []
        for i in batch_data:
            probas = bert_clf(i)
            for item in probas:
                for i in item:
                    answers_temp.append(i)            
        token_answers.append(answers_temp)
        print(answers_temp)
        answers_temp = []
        if count==5:
          break
        try:
          loss_func = nn.MSELoss(reduction = 'sum')
          print(answers_temp.size, batch_probs.size)
          batch_loss = loss_func(torch.tensor(answers_temp), torch.tensor(batch_probs))
          batch_loss.backward()
          optimizer.step()
        except:
          continue
    
      
      

epoch_num ---->0
[0.5097906589508057, 0.4866931438446045, 0.4128294289112091]
[0.5072095394134521, 0.4846435487270355, 0.45699164271354675, 0.5161570310592651, 0.44299373030662537, 0.44648268818855286, 0.4380336403846741, 0.5067108273506165, 0.5168303847312927, 0.5077243447303772]
[0.4802769720554352, 0.48637089133262634, 0.5304993391036987, 0.457534521818161, 0.4928857684135437, 0.4711836576461792, 0.43170252442359924, 0.4164516031742096, 0.4971231520175934]
[0.47517073154449463, 0.45780807733535767]
[0.5229343771934509, 0.4100739061832428, 0.3948374390602112, 0.4990491569042206, 0.50278639793396, 0.485446959733963, 0.452657550573349, 0.5241411328315735, 0.4933161437511444, 0.44707414507865906, 0.48711445927619934, 0.47073113918304443]
epoch_num ---->1
[0.40720757842063904, 0.4803933799266815, 0.48927053809165955]
[0.4160015881061554, 0.4383767247200012, 0.5118116736412048, 0.4929916560649872, 0.47873154282569885, 0.5265690088272095, 0.5134424567222595, 0.43914029002189636, 0.50706189